In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service # 버전 일치!!!!!!! .install() 함수 활용
from selenium.webdriver.chrome.options import Options # 브라우저 설정을 데스크탑/꽉 찬 화면/임시메모리설정/사용하는언어 등등의 옵션 설정 가능
from selenium.webdriver.common.by import By
# 어떤 방식의 명령어로 찾아오게 할 것인지 (셀렉터)
# id가 나오면 찾을 때 땡큐인데 개발자들은 보통 id를 남발하지 않음...
# 그래서 id가 있다면 id로 찾고! 그 다음이 CSS Select(or 클래스) -> XPATH -> 태그(근데 이건 거의 불가능 한 페이지에 div, span태그는 넘 많으니까)
from webdriver_manager.chrome import ChromeDriverManager
# import time
# import re #-> 정규표현식 !!! a 태그(hrep) 없이도 링크 넣은 곳... 정규표현식으로 찾을 수 있음!
import time, re # import time, re, pandas as pd 이렇게 더 묶어서 쓸 수도 있대~!!!!!!!
import pandas as pd

service = Service(ChromeDriverManager().install()) # 최신 버전과 일치시키게 하기 위함!
options = Options() # 많지만 그래봐야 7개! 행운의 7이라고 생각!
# options.add_argument("--headless") # 가장 처음 배웠던 거, 브라우저 띄우지 말고 바로 가! -> 불필요하게 그래픽카드 메모리 낭비하게 됨
# options.add_argument("--disable-gpu") # -> 그래서 필요한 게 이거! headless랑 같이 가는 친구라고 생각 (안 쓸 땐 같이 안 씀)
options.add_argument("--disable-dev-shm-usage") # 리뷰 1,000개 있으면 메모리 너무 많이 차지, 그럴 때 임시메모리 말고 로컬컴퓨터 메모리 사용하게끔
options.add_argument("--start-maximized") # 반응형 페이지에서 꼭 해야 함!!
options.add_argument("--window-size=1920x1080")
options.add_argument(
    "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") # 우리의 어떤 언어가 모국어냐
# 어지간하면 이 정도로 끝남! 크롤러의 준비 운동 끝~~~

driver = webdriver.Chrome(service=service, options=options)
# Chrome() / 대문자로 시작하는데 뒤에 소괄호까지 있다? -> 프로토 타입 객체 (얘를 통해서 하나의 요소? 객체?가 탄생될 것이다~)
# 메뉴얼 - 클래스를 통해 만들어짐 -> 반드시 프로토 타입 객체를 거쳐야 하고 -> 인스턴스 객체가 됨
# 이거 이따가 GPT랑 다시 공부;;

# 의류 - 가방 - 신발 - 액세사리 카테고리 돌 것임 !!! -> 나는 티셔츠,원피스,블라우스로 했는데 더 큰 카테고리였음...ㅋㅋ;;ㅠㅠ

category_urls = {
    "여성 의류":"https://display.wconcept.co.kr/rn/best?displayCategoryType=10101&displaySubCategoryType=ALL&gnbType=Y",
    "가방":"https://display.wconcept.co.kr/rn/best?displayCategoryType=10102&displaySubCategoryType=ALL&gnbType=Y",
    "신발":"https://display.wconcept.co.kr/rn/best?displayCategoryType=10103&displaySubCategoryType=ALL&gnbType=Y",
    "액세서리":"https://display.wconcept.co.kr/rn/best?displayCategoryType=10104&displaySubCategoryType=ALL&gnbType=Y"
}
# 이 상황에서 딕셔너리를 쓰는 이유! -> 'key와 value값이 반복되어야 편하다' 라는 생각이 선생님에겐 있음! but, 우리에겐 없음! => 연습을 덜 한 거야...ㅠ
# 이 딕셔너리를 통해 반복해서 크롤링 시키게 할 거임

all_data = []
# 리스트 형태로 받겠다 (전역변수로 설정해서 각 url안에 있는 데이터들을 저 리스트에 모아줌)

##### 아주아주 중요한 것
# 마우스포인터가 손가락으로 바뀌면 반드시 a태그가 있어야 함! 링크로 들어갈 수 있게
# 근데 wconcept은 a태그가 없음!!!!!! 눈씻고 찾아봐도!!!!!
# >>>>> 노션 참고 <<<<<

# 여기서 gpt 힘 빌렸으면 def가 나왔을 것, or 반복문!
# def로 함수 만드는 게 편함!!!!! 이미지 태그의 307350085 <<< 이런 아이디값만 꺼내오기 위해 ! 이걸 반복할 거니까

def extract_product_id_from_imgsrc(src:str) -> str | None :
    if not src : #  src가 없다면 None (위에 | None을 설정해 줘서 쓸 수 있는듯?...)
        return None
    m = re.search(r"/(\d+)(?:[_\.][^/?]+)?(?:\?.*)?$", src)
    return m.group(1) if m else None
    # 정규표현식 안에 있는 () () 를 각각의 그룹으로 나눌 수 있나봄! 그래서 (\d+)의 값만 가져와서 그걸 m이라는 변수에 넣을 것!
# (src:str) -> str | None ▶ 이미지(태그) 안에 있는 src 정보를 가져올 건데 이 안에 들어올 값은 문자열이야! str로 반환해 (또는 None)
# | (버티컬바) = or
# re. -> 정규표현식 쓰겠다는 뜻
# r"" -> 문자열로 인식시켜주기 위해...!
# src 안에 들어온 문자(src="https://product-image.wconcept.co.kr/productimg/image/img1/95/307335695_HG10377.jpg?RS=412") 안에서!
# (\d+) 여기서 \d는 패턴! 10진수를 찾아와라 / + : 한자리수 이상
# ?: ▶ 찾긴 찾되, 그 값을 가져오지는 마!!!!!
# [_\.] ▶ 언더바 or 온점은 OK야.
#.(온점) 앞에 \(역슬래시) 를 붙인 이유 : .는 정규표현식에서 문자열 하나를 의미함(익스케이프시퀀스?) 그걸 문자열 . 으로 인식시켜주기 위해 \가 필요했던 것임!
# 뒤에 나오는 \? 도 같은 의미!!!
# () 뒤에 ?가 붙어 있으니 이 전체 블록은 옵션(있어도 되고 없어도 됨)
# [^/?]+ ▶ / 또는 ? 이 나오면 안 된다 ( ^이 붙은 여기서는 또 역슬래시로 또는을 안 써도 된대;;;어우)
# 그 뒤에 붙은 " ? " : 있을 수도 있고, 없을 수도 있다는 의미
# (?:\?) ▶ (?:?) 라고 쓰면 에러가 남 ?: 는 문법적 의미라서ㅇㅇ -> \?는 ?를 문자열로 인식시켜주기 위한 역슬래시임
# (?:\?.*) 결론: . ▶ 물음표 뒤에 문자가 올 수 있다 # * ▶ 개수 상관없이
# ?$ 조합 : "문자열이 끝날 때, ?... 같은 쿼리 파라미터가 있을 수도 있고, 아예 없을 수도 있다"
# 암호같음 정규표현식 진짜 어렵ㅜ 깃허브와 맞먹음 그래서 정규표현식을 잘 써야 연봉이 높다

def build_product_url(pid : str | None) -> str : # 프로덕트아이디라는 매개변수에 문자열(307350085) 또는 None이 들어올 것임
    return f"https://www.wconcept.co.kr/Product/{pid}" if pid else "" # 위에서 저 짓을 한 이유가 결국 저 url에 각 상품의 아이디 값(307350085)을 넣기 위함

for cat_name, url in category_urls.items() : # 얘는 4번 반복하는 거임!
    driver.get(url)
    time.sleep(3)

    for _ in range(3) : # 10번 반복해 (각 카테고리마다 10번씩 스크롤한다는 뜻) 나는 짧게 하려고 3번으로 고쳤음
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)

    products = driver.find_elements(By.CSS_SELECTOR, "div.product-item.item.type-all")[:100]
    # CSS 선택자 기준으로 값을 찾아오고 싶어! 100개까지 ---> 클래스 찾는 팁 노션에 정리해둠!

    for product in products :
        try :
            name = product.find_element(By.CSS_SELECTOR, "span.prdc-title span.text.detail").text.strip()

            
            try : # 함수 사용할 거라 에러에 대비해 try문 안에 또 try...^^
                img = product.find_element(By.CSS_SELECTOR, "button.area-img img") # area-img라는 버튼 안에 있는 img태그를 찾아오겠다는 거임
                src = img.get_attribute("src")
                # 이미지 안에 src="https://product-image.wconcept.co.kr/productimg/image/img1/85/307350085_GG98433.jpg?RS=412" 이걸 찾아오라는 뜻임
                pid = extract_product_id_from_imgsrc(src) # 위에서 만든 함수 쓰는 거 (아이디 값(307350085) 찾으려구)
                url = build_product_url(pid) # 위에서 만든 함수 쓰는 거 그 아이디값을 url 맨 뒤에 넣으려구

                
            except :
                url = "" # 에러를 보고 싶지 않아서 그냥 url값을 비운 채로 진행하곘다는 뜻

            try :
                review_info = product.find_element(By.CSS_SELECTOR, "span.area-info span.text.cnt").text
                # 스트립 안 해도 됨! 정규표현식으로 숫자만 추출해낼거야
                review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0
                # 정규표현식은 기본적으로 문자열에서만 사용 가능 (숫자가 아닌 값을 찾아서 없애주기 위해 ^(캐럿)이 대괄호 안으로 들어감)
                # 그 자체를 정수로 변경해주기 위해!!! int() 함수 사용! int 하지 않으면 숫자로 보여도 문자열의 상태라서 SQL로 넘어갈 때 에러 발생
                # if review_info else 0 ▶ review_info 가 있다면 실행하고, 없다면 0 으로 표시
            except :
                review_count = 0

            try : 
                rating_text = product.find_element(By.CSS_SELECTOR, "span.area-info em.score").text #
                review_avg = float(re.findall(r"[0-9.]+", rating_text)[0])
                # 0-9 뒤에 . -> 알아서 소수점으로 인식 (여기서는 저 .(온점)을 문자로 인식시켜주기 위해서 \ 역슬래시 쓰는 거 아님 ㄴㄴㄴ)
                # [0-9.] 자체가 하나의 정해진 문법이라 [0-9]를 \d로 쓰면 안 된다
                # 여러 상황에서도 적용될 수 있게끔 만들어주신 거임...! 원래 이 사이트 상에서는 그냥 em태그만 찾아도 바로 가져올 수 있긴 함
            except :
                review_avg = 0.0

            all_data.append({
                "카테고리": cat_name,
                "상품명": name,
                "URL": url,
                "리뷰 수": review_count,
                "평점": review_avg
            })

        except Exception as e :
            print("에러 발생 :", e)
            continue

driver.quit()

# 딕셔너리 (key:value 중 value 값만 가져와야 함 그때 쓰는 함수 : items)
# 카테고리이름은 cat_name으로, url은 url로.
# 크롤러 driver를 이용!
# 스크롤 내리기 위해 중첩 for문 활용!
# .execute_script : 자바스크립트의 언어를 사용하겠다는 함수 
# "window.scrollTo(0, document.body.scrollHeight)"
# 스크롤 10번 하는 이유 -> 안전장치라고 생각. 페이지 스크롤이 언제 바뀔지 모르닝 (ex.인스타 같은 경우에는 꼭 필요!)

# 에러에 대비, 안전장치로 try문 안에서 크롤링 하는 게 좋음
# except Exception as e 에러 생기면 Exception 라는 변수 발생


# print(all_data)로  데이터 잘 나오는지 확인 후
# ------ 이제 pandas... ------- #

df = pd.DataFrame(all_data) # all_data 라는 리스트에 담긴 값을 행,열 맞게 변환! # df = DataFrame의 약자
# 🔻 카테고리와 무관하게 리뷰수 많은 순으로 100개만 끊어내고 싶을 때! sort_values() 특정 컬럼 기준 정렬 함수

df_top100 = df.sort_values(by="리뷰 수", ascending=False).head(100).reset_index(drop=True)
# ascending=False ▶ 내림차순
# .reset_index(drop=True) 정렬된 이후의 index값으로 reset 하고 싶어! 기존 인덱스는 drop (버려라)

category_summary = df.groupby("카테고리").agg({  # 카테고리를 기준으로! groupby 여기서도 쓸 수 있음 # agg : 집계함수를 쓰겠다(sum)라는 함수
    "리뷰 수":"sum", # 카테고리별로 리뷰수를 알아서 더해줌
    "평점":"mean",
}).reset_index()
# 여기서는 (drop=True) 쓰면 안 됨 왜냐면 카테고리가 사라짐!!!
# 🔻 이렇게 프린트 됨 그냥 리뷰수 평점... 뭔 카테고리인지 모름
# 카테고리별 통계 :
#     리뷰 수     평점
# 0  24218  4.267
# 1  27342  4.281
# 2  30519  4.016
# 3  31520  4.435

print(f"""
상위 100개 상품 :
{df_top100.head()}

카테고리별 통계 :
{category_summary}
""")


상위 100개 상품 :
    카테고리                                    상품명  \
0     신발                OORIGINAL BLACK - 조리 블랙   
1     가방  [14%쿠폰] [단독] Panier Petit Bag_6colors   
2   액세서리   [단독] Round Logo Ball Cap (10 Colors)   
3   액세서리           [단독]BT8032 티디 스퀘어 버클 벨트 18mm   
4  여성 의류                       실켓 코튼 티셔츠_8color   

                                            URL  리뷰 수   평점  
0  https://www.wconcept.co.kr/Product/301721294  8478  5.0  
1  https://www.wconcept.co.kr/Product/301756891  3730  5.0  
2  https://www.wconcept.co.kr/Product/301628486  3682  4.9  
3  https://www.wconcept.co.kr/Product/304460283  3131  4.9  
4  https://www.wconcept.co.kr/Product/301218440  2968  4.9  

카테고리별 통계 :
    카테고리   리뷰 수     평점
0     가방  23521  4.236
1     신발  29208  4.284
2   액세서리  24527  4.299
3  여성 의류  28968  4.436



In [ ]:
# 엑셀로 보내기

In [19]:
from openpyxl import Workbook

wb = Workbook()

ws_top100 = wb.active
ws_top100.title = "상위100상품" # 이게 진짜 시트의 이름!

ws_top100.append(["카테고리","상품명","URL","리뷰 수","평점"]) # 첫 번째 행은 이렇게 다섯 개의 컬럼!

for idx, row in df_top100.iterrows() :  # .iterrows() : df 안에 있는 각각의 행을 하나씩 찾아와서 넣어줄 수 있음 [카테고리/상품명/url/리뷰 수/평점]
    ws_top100.append([row["카테고리"], row["상품명"], row["URL"], row["리뷰 수"], row["평점"]])

ws_category = wb.create_sheet(title="카테고리별 통계") # 아예 또다른 새로운 시트를 만드는 거라서 .create_sheet() 사용 !

ws_category.append(["카테고리", "총 리뷰 수", "평균 평점"]) # 얘는 첫 번째 행이고 그 아래 행에 차례대로 들어갈 데이터를 넣을 for문 만들 거임

for idx, row in category_summary.iterrows() : # 위에서 만든 거! df 말고 통계 데이터임 4개의 행이 있으니까 그대로 들어가곘군!
    ws_category.append([
        row["카테고리"], row["리뷰 수"], round(row["평점"], 2)
    ])
    # round 함수로 반올림하여 소수 두번째자리까지 나오게끔

filename = "wconcept_result_v1.xlsx"
wb.save(filename)
# filename 변수 지정 안 하고 바로 wb.save("wconcept_result.xlsx") 해도 ㄱㅊ 그게 원래 우리가 하던 방식임
print(f"✅ 1개의 엑셀 파일에 시트 2개 저장 완료 : {filename}")
#wb.close

✅ 1개의 엑셀 파일에 시트 2개 저장 완료 : wconcept_result_v1.xlsx


In [ ]:
# sql 로 보내기 !

In [29]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text # create_engine : 커밋까지 자동화시키기 가능! # text : 미리 만들어둔 변수 값을 테이블로 만들어줄 수 있음
from urllib.parse import quote_plus # 얘가 있어야 됨 비번이슈

DB_NAME = "wconcept_db_v2"
TABLE_TOP100 = "top100_products"
TABLE_CATEGORY = "category_summary"
USER = "root"
PASSWORD_RAW = "Tnalstn0525@" # conn에서는 이걸로 비번 입력하고!!!
PASSWORD = quote_plus(PASSWORD_RAW) # sqlalchemy에서는 나는 @ 때문에 얘로 인코딩 시켜서 사용해야 함
# 위에 from urllib.parse import quote_plu 도 해 줘야 하고!!!!!
HOST = "localhost" # or "127.0.0.1"
PORT = 3306

xlsx_path = "wconcept_result_v1.xlsx"
SHEET_TOP100 = "상위100상품" # 위에서 만든 엑셀 시트 이름 적기
SHEET_CATEGORY = "카테고리별 통계"

df_top100 = pd.read_excel(xlsx_path, sheet_name=SHEET_TOP100)
df_category = pd.read_excel(xlsx_path, sheet_name=SHEET_CATEGORY)

# 🔻 테이블 컬럼명을 바꾸기 위한...것... 한글->영어
top100_cols = {
    "카테고리":"category", 
    "상품명":"product_name", 
    "URL":"url", 
    "리뷰 수":"review_count", 
    "평점":"avg_rating"
}

category_cols = {
    "카테고리":"category",
    "총 리뷰 수":"total_review_count",
    "평균 평점":"avg_rating"
}

df_top100 = df_top100.rename(columns=top100_cols)[list(top100_cols.values())]
df_category = df_category.rename(columns=category_cols)[list(category_cols.values())]
# df_top100이라는 데이터를 찾아와서 컬럼값의 필드값인 5개의 한글 문자열을 위에서 정의한 영어 문자열로 바꿔라
# 실제로 바꾸는 건 [list()] <-- 여기서! / 엑셀에 넣을 때도 대괄호 안에 값을 넣어서 여기서도 대괄호 안에 넣어랑
# [list(category_cols.values)] key:value 중 value 값들로 이루어진 리스트인 거여

# 🔻 재할당 (나중에 제대로 배울 거니까 오늘은 보기만 해도 됨)
df_top100 = df_top100.dropna(subset=["category", "product_name", "review_count", "avg_rating"])
df_category = df_category.dropna(subset=["category", "total_review_count", "avg_rating"])
# dropna : not a number?? no answer?? -> 값이 없는 애들(null)은 버리겠다는 뜻이래
# subset= 이 안에 검사하고자 하는 값을 넣어주면 됨
# url이 빠진 이유? url은 무조건 값이 있기 때문임...! 근데 안전하게 하려고 걍 다시 넣음 (아놔 ㅋ) 다시 뺌 아으악 ;;;

# 🔻 자료형 (크롤링 할 때도 변환을 시켰지만 엑셀로 들어가면서 그 데이터가 문자열로 인식될 수도 있기 때문에 sql로 넘어갈 때 또 변환을 시켜줘야 함)
df_top100["review_count"] = df_top100["review_count"].astype(int) # .astype(int) 정수값으로 들어와야 한다는 의미
df_top100["avg_rating"] = df_top100["avg_rating"].astype(float).round(2) # 소수점 2번째자리에서 반올림한 실수값

df_category["total_review_count"] = df_category["total_review_count"].astype(int) # .astype(int) 정수값으로 들어와야 한다는 의미
df_category["avg_rating"] = df_category["avg_rating"].astype(float).round(2) # 소수점 2번째자리에서 반올림한 실수값

conn_admin = pymysql.connect(host=HOST, user=USER, password=PASSWORD_RAW, port=PORT, autocommit=True)
# 그냥pymysql은 connet 후 cursor를 만들어서 그 안에서 직접 테이블 만들고 커밋까지 다 했어야 함
# autocommit=True ▶ 커밋 자동화
# conn_admin 안에 저 커넥트 요소를 넣어서 @@@@@@@@@@@

with conn_admin.cursor() as cur :
    cur.execute(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
    # cur.execute : 데이터베이스 만들 수 있는 권한 준 것 # {DB_NAME} : 맨 위에서 선언한 거
    # CHARACTER SET utf8mb4 : 한글/특수문자 등이 인코딩/디코딩 과정에서 이상한 문자로 깨질 가능성을 보완
    # COLLATE utf8mb4_unicode_ci : 이모지/특수문자 허용하겠다??
    # {DB_NAME} 앞뒤로 백틱(`) 넣으면 안전장치! 특수문자 (_) 때문에 오류날 수도 있어서

conn_admin.close() # DB 까지 만들고 종료한 것!

# ---- ▲ 여기까지는 설계 단계! ----- 실제 실행은 create_engine 으로

engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
)
# 얘는 그냥 문법이라... 걍 외우거나 쓰려고 할 때 복붙... 내 정보를 넣는 문법인가봐
# 이게 있어야 엔진을 구현할 수 있다는 약속 뭐 그런 거 ...

ddl_top100 = f"""
CREATE TABLE IF NOT EXISTS `{TABLE_TOP100}` (
    `id` BIGINT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT, 
    `category` VARCHAR(50) NOT NULL,
    `product_name` VARCHAR(255) NOT NULL,
    `url` TEXT NULL,
    `review_count` INT NOT NULL, 
    `avg_rating` DECIMAL(3,2),
    `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
)
"""
# ddl : 데이터데피니션랭귀지?
# `id` : `` (백틱)을 통해 이게 아이디값이라는 구분 지어줌!
# NULL : 비어있을 수도 있다.!
# DECIAL(3,2) : 소수자리까지 합쳐서 총 3자리, 소수는 2번째자리까지 나오게끔 (그래서 정수는 한자리임!) 4.07 이런 식

ddl_category = f"""
CREATE TABLE IF NOT EXISTS `{TABLE_CATEGORY}` (
    `id` INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `category` VARCHAR(50) NOT NULL,
    `total_review_count` INT NOT NULL,
    `avg_rating` DECIMAL(3,2),
    `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
)
"""



with engine.begin() as conn :
    conn.execute(text(ddl_top100))
    conn.execute(text(ddl_category))
# engine.begin() ▶ engine을 사용해서 무언가를 시작하곘다
# text() : 이 안에 있는 변수를 실행시킨다 ! (MySQL에서 ddl_top100 얘(안에있는문자열)를 읽어들여서 실행시키겠다)
# ▲ 결과적으로 저 구문은 테이블 만드는 거얏

with engine.begin() as conn : # 🔻 df_top100 안에 들어갈 하나의 행의 순서를 세팅!(정의)
    df_top100[["category","product_name","url","review_count","avg_rating"]].to_sql(
        TABLE_TOP100, con=conn, if_exists="append", index=False
    )
    df_category[["category","total_review_count","avg_rating"]].to_sql(
        TABLE_CATEGORY, con=conn, if_exists="append", index=False
    )
# df_top100 자체가 리스트형태를 띄고 있어서 []를 쓰고 그 안에 하나의 행이라는 뜻으로 또 []를 적은 것임
# .to_sql() 메서드 함수를 통해 
# con=conn (contents는 위에 들어갈 conn(connet)
# if_exists="append" ▶ 기존에 값이 있다면 걍 추가하겠다 ???
# index=False ▶ index는 자동증가로 이미 있으니까 여기서 굳이 넣지 않겠다 ???

# 🔻 함수 정의 (누락되거나 어떤 오류에 의해 저장되지 않았을 때를 대비... 수동으로 넣기 위한 준비인가??
def ensure_index(conn, schema, table, index_name, column_list) : 
    q = text("""
        SELECT 1
        FROM information_schema.statistics
        WHERE table_schema=:schema AND 
        table_name=:table AND
        index_name=:index_name
    """)
    # text() 함수를 사용한다는 건 그 안에 쿼리 구문을 쓰겠다는 뜻임 ! (sql에서 실행할)
    # information_schema.statistics : schema(DB가 모여져 있는 것)에 있는 일반적인 정보? 에 1이라는 값이 있음
    # MySQL에 있는 기본 환경 정보 같은 거임... ㅎ...몰라
    # table_schema=:schema ▶ 우리가 만든 테이블이 존재하고 # =: 도 그냥 구문임..
    # table_name=:table ▶ 테이블 이름이 일치하고
    # index_name=:index_name ▶ index_name 도 일치한다면...?
    # => 1이라는 값을 가져와라
    exists = conn.execute(q, {"schema":schema, "table":table, "index_name":index_name}).fetchone()

with engine.begin() as conn :
    ensure_index(conn, DB_NAME, TABLE_TOP100, f"idx_{TABLE_TOP100}_category", ["category"])
    ensure_index(conn, DB_NAME, TABLE_TOP100, f"idx_{TABLE_TOP100}_review_count", ["review_count"])
    ensure_index(conn, DB_NAME, TABLE_TOP100, f"idx_{TABLE_TOP100}_avg_rating", ["avg_rating"])
    ensure_index(conn, DB_NAME, TABLE_CATEGORY, f"idx_{TABLE_CATEGORY}_total_review", ["total_review_count"])
    ensure_index(conn, DB_NAME, TABLE_CATEGORY, f"idx_{TABLE_CATEGORY}_avg_rating", ["avg_rating"])

print("✅ MySQL 데이터 생성 완료")

✅ MySQL 데이터 생성 완료


### 🚩 오늘의 미션!!!
- 오늘, 지금까지 작업했던 W-Concept 데이터에서 **리뷰 데이터를 추가하여 SQL 데이터를 업데이트**하세요.
- 그리고, 리뷰값 중 임의로 3개를 선정하여, 해당 문자열 하나로 합쳐서 출력할 수 있도록 쿼리문을 작성하세요!

In [ ]:
# 리뷰 데이터로 이루어진 엑셀 파일 만들기
# 상품명에 맞게 넣는 방법. ..?????
# ㄴㄴㄴㄴㄴ -> 엑셀에 또다른 시트로 만들어준 다음 테이블을 아예 하나 새로 만들기! (상품명과 리뷰 내용만 들어간 테이블로 ㅇㅇ)
# -> 헷갈릴 수 있으니까 요거는 새로운 파일에서 작업하겠음!!!!!!